### Import libraries, metadata

In [ ]:
import os
import librosa
from librosa.filters import mel as librosa_mel_fn
import pickle as pkl
import IPython.display as ipd
from tqdm.notebook import tqdm
import torch
import codecs
import matplotlib.pyplot as plt
%matplotlib inline

from g2p_en import G2p
from text import *
from text import cmudict
from text.cleaners import custom_english_cleaners
from text.symbols import symbols

# Mappings from symbol to numeric ID and vice versa:
symbol_to_id = {s: i for i, s in enumerate(symbols)}
id_to_symbol = {i: s for i, s in enumerate(symbols)}

csv_file = '/media/disk1/lyh/LJSpeech-1.1/metadata.csv'
root_dir = '/media/disk1/lyh/LJSpeech-1.1/wavs'
data_dir = '/media/disk1/lyh/LJSpeech-1.1/espnet2'

g2p = G2p()
metadata={}
with codecs.open(csv_file, 'r', 'utf-8') as fid:
    for line in fid.readlines():
        id, _, text = line.split("|")
        
        clean_char = custom_english_cleaners(text.rstrip())
        clean_phone = []
        for s in g2p(clean_char.lower()):
            if '@'+s in symbol_to_id:
                clean_phone.append('@'+s)
            else:
                clean_phone.append(s)
        
        metadata[id]={'char':clean_char,
                     'phone':clean_phone}

In [ ]:
def text2seq(text):
    sequence = [symbol_to_id[c] for c in text]
    # Append EOS token
    sequence.append(symbol_to_id['~'])
    return sequence

### STFT

In [ ]:
def stft(x,
         n_fft,
         n_shift,
         win_length,
         window='hann',
         center=True,
         pad_mode='reflect'):
    x = x.astype(np.float32)

    # FIXME(kamo): librosa.stft can't use multi-channel?
    # x: [Time, Channel, Freq]
    x = librosa.stft(
        x,
        n_fft=n_fft,
        hop_length=n_shift,
        win_length=win_length,
        window=window,
        center=center,
        pad_mode=pad_mode).T

    return x

def stft2logmelspectrogram(x_stft,
                           fs,
                           n_mels,
                           n_fft,
                           fmin=None,
                           fmax=None,
                           eps=1e-10):
    # x_stft: (Time, Freq)
    fmin = 0 if fmin is None else fmin
    fmax = fs / 2 if fmax is None else fmax

    spc = np.abs(x_stft)
    mel_basis = librosa.filters.mel(fs, n_fft, n_mels, fmin, fmax)
    lmspc = np.log10(np.maximum(eps, np.dot(mel_basis, spc.T)))

    return lmspc

def logmelspectrogram(x,
                      fs=22050,
                      n_mels=80,
                      n_fft=1024,
                      n_shift=256,
                      win_length=None,
                      window='hann',
                      fmin=80,
                      fmax=7600,
                      eps=1e-10,
                      pad_mode='reflect'):
    # stft: (Time, Channel, Freq) or (Time, Freq)
    x_stft = stft(x, n_fft=n_fft, n_shift=n_shift, win_length=win_length,
                  window=window, pad_mode=pad_mode)

    return stft2logmelspectrogram(x_stft,
                                  fs=fs,
                                  n_mels=n_mels,
                                  n_fft=n_fft,
                                  fmin=fmin,
                                  fmax=fmax,
                                  eps=eps)

In [ ]:
from stft import STFT

class TacotronSTFT(torch.nn.Module):
    def __init__(self,
                 filter_length=1024,
                 hop_length=256,
                 win_length=1024,
                 n_mel_channels=80,
                 sampling_rate=22050, 
                 mel_fmin=0.0,
                 mel_fmax=8000.0):
        super(TacotronSTFT, self).__init__()
        self.stft_fn = STFT(filter_length, hop_length, win_length)
        mel_basis = librosa_mel_fn(sampling_rate,
                                   filter_length,
                                   n_mel_channels,
                                   mel_fmin,
                                   mel_fmax)
        self.mel_basis = torch.from_numpy(mel_basis).float()

    def wav_to_specs(self, y):
        assert(torch.min(y.data) >= -1)
        assert(torch.max(y.data) <= 1)

        magnitudes, _ = self.stft_fn.transform(y)
        mel_output = torch.matmul(self.mel_basis, magnitudes)
        melspec = torch.log(mel_output)
        
        return melspec

stft = TacotronSTFT()

In [ ]:
from kaldiio import ReadHelper

datasets = ['char_train_no_dev', 'char_dev', 'char_eval']

espnet = {}
for dataset in datasets:
    feat_path = f'/home/lyh/anaconda3/envs/espnet/egs/ljspeech/tts1/dump/{dataset}'
    for i in range(1, 33):
        with ReadHelper(f'ark:{feat_path}/feats.{i}.ark') as reader:
            for a in reader:
                espnet[a[0]]=a[1]

### Others

In [ ]:
def get_mel(filename):
    wav, sr = librosa.load(filename, sr=22050)
    melspec = logmelspectrogram(wav)
    return torch.from_numpy(melspec), wav


def save_file(fname):
    wav_name = os.path.join(root_dir, fname) + '.wav'
    text = metadata[fname]['char']
    char_seq = torch.LongTensor( text2seq(metadata[fname]['char']) )
    phone_seq = torch.LongTensor( text2seq(metadata[fname]['phone']) )
    
    melspec, wav = get_mel(wav_name)
    #melspec = torch.from_numpy(espnet[fname].T)
    
    with open(f'{data_dir}/char_seq/{fname}_sequence.pkl', 'wb') as f:
        pkl.dump(char_seq, f)
    with open(f'{data_dir}/phone_seq/{fname}_sequence.pkl', 'wb') as f:
        pkl.dump(phone_seq, f)
    with open(f'{data_dir}/melspectrogram/{fname}_melspectrogram.pkl', 'wb') as f:
        pkl.dump(melspec, f)
    
    return text, char_seq, phone_seq, melspec

### Save and Inspect Data

In [ ]:
for k in tqdm(metadata.keys()):
    text, char_seq, phone_seq, melspec = save_file(k)
    if k == 'LJ001-0019':
        print("Text:")
        print(text)
        print()
        print("Melspectrogram:")
        plt.figure(figsize=(16,4))
        plt.imshow(melspec, aspect='auto', origin='lower')
        plt.show()